# Reconocimiento de caras con Keras usando una VGG Face. 

In [ ]:
# Este codigo nos permite comparar imagenes con caras y determinar si son de la misma persona o no.


In [ ]:
# Imports necesarios

import sys
import numpy as np
from PIL import Image
from scipy import misc, ndimage
from keras import Model
from keras import Sequential
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import ZeroPadding2D
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.models import model_from_json
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import preprocess_input


# La imagen hay que procesarla para adaptarla a la red VGG-FACE


def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img
 
# DEFINIMOS el modelo de la VGG-FACE
    
def prepare_model():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
     
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
     
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
     
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
     
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
     
    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    
# Cargamos los pesos de la VGG-FACE

    model.load_weights('vgg_face_weights.h5')
    
# Modificamos el clasificador anulando las dos ultimas capas lo que nos devolvera un vector
#que podra ser comparado con otros vectores (imagenes) utilizando para ello
#la metrica similitud de coseno

    vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)
    return vgg_face_descriptor


# Funcion que devuelve la simitud entre dos imagenes.



def same_person(model, img1, img2):
    img1_representation = model.predict(preprocess_image(img1))[0,:]
    img2_representation = model.predict(preprocess_image(img2))[0,:]

    a = np.matmul(np.transpose(img1_representation), img2_representation)
    b = np.sum(np.multiply(img1_representation, img1_representation))
    c = np.sum(np.multiply(img2_representation, img2_representation))
    cosine_similarity = 1 - (a / (np.sqrt(b) * np.sqrt(c)))
    print(cosine_similarity)

    if cosine_similarity < 0.4:
        return True
    return False

model = prepare_model()

# Esta funcion debera ser modificada y hacer otra si lo que pretendemos es tomar
# una foto como referencia , que sea comparada con el esto de fotos de un album, carpeta,etc y se retorne todas aquellas fotos 
#que pertenezcan a la foto de referencia

# Crear funcion: 

if same_person(model, 'bengio1.jpg', 'bengio2.jpg'):
    print("Same person!")
else:
    print(":(")